# Rename core photos

---

<img src="img/corerenameweb.jpg" width="800" />

---

## How to use

Prerequisite:
- All images must be copied on a web folder somewhere on the Internet, accessible with a browser (No Dropbox or other like it...)
- In the same folder a file named `list.txt` must have the list of photos, one filename per line.

1. Run the cell below (click it and hit Shiht-Enter)
2. Copy and paste the URL where the images are stored, and click `Open`
3. The first photo will display along with its name and the total number of photos
3. Fill-in the info for that photo and click `Rename photo`
3. A MSDOS batch file is created below the image
4. Repeat step 4...
4. When complete, copy the content of the bottom textarea (batch file)
5. Paste its content in a text file and save as `rencore.bat` in the same **local** folder where the photos are
6. Double-click `rencore.bat` to rename all photos in the local folder

Renaming is trying to get smart:

- Only Dry/Wet state will change between two consecutive photos if `2 photos per interval` is checked
- `Box number` increment by one from one photo to the next
- `mFrom` is the previous `mTo` from one photo to the next

To skip a photo and go to the next one, click: `Skip photo`.

You can stop at any time, just close the Notebook.

You can also restart at any time by running (Shiht-Enter) the cell again

## Run hidden cell below:

In [ ]:
import os
import urllib.request
import ipywidgets as widgets

photolst = []
nphoto = 0
dw = 0
spcs = '&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; |&nbsp;&nbsp;&nbsp;&nbsp;'


# ----------------------------------------------------------------
def f(v):
    """Format number."""
    #
    return '%0.2f' % v


def nextphoto(inc=True):
    """Display next photo."""
    #
    global nphoto
    #
    if inc:
        nphoto += 1
    if nphoto >= len(photolst):
        nphoto = 0
        labl.value = "<b><font color='red'>WARNING: This is the last photo!</font></b>"
        shownp(0)
        return '<img src="' + photolst[-1] + '" width="900" />'
    else:
        labl.value = "<font color='black'>%s</font>" % photolst[nphoto]
        return '<img src="' + photolst[nphoto] + '" width="900" />'


def donext():
    """Change display for next photo."""
    #
    mfrom.value = mto.value
    boxnb.value += 1


def saveren(L):
    """Save current rename operation."""
    #
    bat.value += 'ren ' + photolst[nphoto].split('/')[-1] + ' ' + L + '\n'


def shownp(n):
    """Show renaming progress."""
    #
    labn.value = spcs + 'Number of photos to rename: ' + str(n) + '/' + str(len(photolst))


# Callbacks
def openurl(btn):
    ''' Return a string with the HTML content from url, using a Mozilla/5.0 user-agent. '''
    #
    global photolst, nphoto
    #
    url = theurl.value + '/list.txt'
    try:
        req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        tmp = urllib.request.urlopen(req).read()
    except:
        tmp = b''
    #
    thelist = str(tmp, 'utf-8').split('\n')
    photolst = [theurl.value + '/' + f for f in thelist]
    if photolst[-1] == '':
        _ = photolst.pop()
    nphoto = 0
    photo.value = nextphoto(False)
    shownp(len(photolst))
    bat.value = '@echo off\n'


def rename_photo(btn):
    """Rename the current photo and jump to next one."""
    #
    global nphoto, dw
    #
    # Rename
    new_name = str(holeid.value).strip() + '_' + f(mfrom.value) + '-' + f(mto.value) + 'm_Box' + str(boxnb.value) + '_'
    new_name += 'D' if drywet.value == 1 else 'W'
    new_name += os.path.splitext(photolst[nphoto])[1]
    saveren(new_name)
    #
    # Setup new info
    if dw == 0 and twoimage.value:
        drywet.value = 2 if drywet.value == 1 else 1
        dw = 1
    elif twoimage:
        drywet.value = 2 if drywet.value == 1 else 1
        donext()
        dw = 0
    else:
        donext()
    #
    photo.value = nextphoto()
    shownp(len(photolst) - nphoto)


def to_next(btn):
    """Skip to next photo."""
    #
    global nphoto
    #
    photo.value = nextphoto()


# ----------------------------------------------------------------
# Define widgets
labu = widgets.HTML(
    value='Core image URL:',
    layout=widgets.Layout(width='200px'))
theurl = widgets.Text(
    value='',
    placeholder='type in a URL',
    description='',
    layout=widgets.Layout(width='600px'),
    disabled=False
)
btnurl = widgets.Button(
    description='Open',
    disabled=False,
    button_style='',
    tooltip='Click to open ural and read list.txt',
)
btnurl.on_click(openurl)


labl = widgets.HTML(value=' ')
labn = widgets.HTML(value=' ')
twoimage = widgets.Checkbox(
    value=True,
    description='2 photos per interval',
    disabled=False,
    indent=False
)
coretype = widgets.Dropdown(
    options=[('Whole core', 1), ('Half core', 2), ('Quarter Core', 3)],
    value=1,
    description='Core Type:',
)
drywet = widgets.Dropdown(
    options=[('Dry', 1), ('Wet', 2)],
    value=1,
    description='Dry or Wet:',
)
holeid = widgets.Text(
    value='',
    placeholder='Hole ID',
    description='Hole ID:',
    disabled=False
)
boxnb = widgets.BoundedIntText(
    value=1,
    min=1,
    max=9000,
    step=1,
    description='Box number:',
    disabled=False
)
mfrom = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=10000,
    step=3,
    description='mFrom:',
    disabled=False
)
mto = widgets.BoundedFloatText(
    value=3,
    min=0,
    max=10000,
    step=3,
    description='mTo:',
    disabled=False
)
rena = widgets.Button(
    description='Rename photo',
    disabled=False,
    button_style='',
    tooltip='Click to rename photo',
    icon='camera-rotate'
)
skp = widgets.Button(
    description='Skip photo',
    disabled=False,
    button_style='',       # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to skip this photo',
    icon='angles-right'    # (FontAwesome names without the `fa-` prefix)
)
photo = widgets.HTML(value="<h1>Welcome to core photo renamer!</h1>")
bat = widgets.Textarea(
    value='',
    layout = widgets.Layout(width='650px'),
    disabled=False
)

# Setup callbacks
rena.on_click(rename_photo)
skp.on_click(to_next)

# Setup UI
main_layout = widgets.Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    border=None,
    width='100%'
)
box_layout = widgets.Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    border=None,
    width='450px'
)
col_layout = widgets.Layout(
    display='flex',
    flex_flow='row',
    align_items='stretch',
    width='900px',
    border='solid 1px',
    padding='3px'
)
btn_layout = widgets.Layout(
    display='flex',
    flex_flow='row',
    align_items='stretch',
    border='solid 1px',
    padding='3px',
    margin='auto'
)
lbl_layout = widgets.Layout(
    display='flex',
    flex_flow='row',
    width='900px',
    align_items='stretch'
)

urlly = widgets.Box(children=[labu, theurl, btnurl], layout=lbl_layout)
labls = widgets.Box(children=[labl, labn], layout=lbl_layout)
btns = widgets.Box(children=[rena, skp], layout=btn_layout)
litems = [twoimage, holeid, mfrom, mto]
ritems = [btns, boxnb, coretype, drywet]
lbox = widgets.Box(children=litems, layout=box_layout)
rbox = widgets.Box(children=ritems, layout=box_layout)
bbox = widgets.Box(children=[lbox, rbox], layout=col_layout)

box = widgets.Box(children=[urlly, labls, bbox, photo, bat], layout=main_layout)
box

In [ ]:
#  https://www.geoproc.com/K/cores